#### Get all the necessary credentials

Create a function that locates the respective files (those need to be saved in the Credentials folder):

In [1]:
from pathlib import Path
import os

def locate(filename):
    cwd = Path(os.getcwd())
    cwd_new = str(cwd.parent)+'\Credentials'
        
    for r,d,f in os.walk(cwd_new):
        for files in f:
            if files == filename:
                path = os.path.join(r,files)
              
    with open(path, 'r') as f:
        content = f.readlines()
        content = [x.strip() for x in content]
        
    return(content)

#### Get your current location

In [2]:
from geopy.geocoders import GoogleV3

address = locate('home_address.txt')[0]
key = locate('login_googleAPI.txt')[0]

geolocator = GoogleV3(api_key=key)
current_latlon = geolocator.geocode(address)[1]

#### Identify the available bicing stations within the area

Select the distance you are prepared to walk:

In [3]:
distance = 0.3 # in kilometers

Get the nearest stations within the radius:

In [4]:
import json
import requests

current_lat = current_latlon[0]
current_lon = current_latlon[1]

url = 'http://barcelonaapi.marcpous.com/bicing'
url_nearest = url+'/nearstation/latlon/'+str(current_lat)+'/'+str(current_lon)+'/'+str(distance)+'.json'
res_nearest = requests.get(url_nearest)
json_nearby = json.loads(res_nearest.text)

Create a list of the selected stations:

In [5]:
station_id = []
station_name = []
station_lat = []
station_lon = []
distance = []

for idx,i in enumerate(json_nearby['data']['nearstations']):
    station_id.append(int(json_nearby['data']['nearstations'][idx]['id']))
    station_name.append(json_nearby['data']['nearstations'][idx]['name'])
    station_lat.append(float(json_nearby['data']['nearstations'][idx]['lat']))
    station_lon.append(float(json_nearby['data']['nearstations'][idx]['lon']))
    distance.append(float(json_nearby['data']['nearstations'][idx]['distance']))

Find out how many bikes are available at each of the selected stations:

In [6]:
import datetime

bikes = []
slots = []
dates = []
bikes_ok = []
slots_ok = []

for idx,i in enumerate(station_id):
    url_availability = 'http://wservice.viabicing.cat/v2/stations/'+str(i)
    res_availability = requests.get(url_availability)
    json_availability = json.loads(res_availability.text)
    
    bikes_num = json_availability['stations'][0]['bikes']
    slots_num = json_availability['stations'][0]['slots']
    
    bikes.append(bikes_num)
    slots.append(slots_num)
    
    if int(bikes_num) >= 2:
        bikes_ok.append(True)
    else:
        bikes_ok.append(False)
        
    if int(slots_num) >= 2:
        slots_ok.append(True)
    else:
        slots_ok.append(False)
            
    dates.append(datetime.datetime.fromtimestamp(json_availability['updateTime']).strftime('%Y-%m-%d %H:%M:%S'))

Create a dataframe so that we can easily filter out the non-available stations:

In [7]:
import pandas as pd
dataset = pd.DataFrame({'station_id':station_id,
                        'station_name':station_name,
                        'station_lat':station_lat,
                        'station_lon':station_lon,
                        'bikes':bikes,
                        'slots':slots,
                        'bikes_ok':bikes_ok,
                        'slots_ok':slots_ok,
                        'entry_date':dates
                       })
dataset

,bikes,bikes_ok,entry_date,slots,slots_ok,station_id,station_lat,station_lon,station_name
0,0,False,2018-07-04 11:21:08,20,True,29,41.401101,2.170082,"Provença, 388"
1,17,True,2018-07-04 11:21:08,11,True,19,41.403282,2.170726,"Rossello, 354"
2,1,False,2018-07-04 11:21:08,30,True,362,41.397006,2.170570,"Bailén, 100"
3,1,False,2018-07-04 11:21:08,18,True,123,41.397411,2.167798,"Girona, 126"
4,0,False,2018-07-04 11:21:08,24,True,224,41.399102,2.165556,"Girona, 156"
5,2,True,2018-07-04 11:21:08,27,True,370,41.403735,2.172987,"Sardenya, 234"
6,0,False,2018-07-04 11:21:08,19,True,22,41.402051,2.175206,"Sardenya, 296"


#### Map out the available stations

Filter out stations that are not available:

In [8]:
dataset_clean_bikes = dataset[dataset.bikes_ok == True].reset_index(drop=True)
dataset_clean_bikes

,bikes,bikes_ok,entry_date,slots,slots_ok,station_id,station_lat,station_lon,station_name
0,17,True,2018-07-04 11:21:08,11,True,19,41.403282,2.170726,"Rossello, 354"
1,2,True,2018-07-04 11:21:08,27,True,370,41.403735,2.172987,"Sardenya, 234"


In [9]:
dataset_clean_slots = dataset[dataset.slots_ok == True].reset_index(drop=True)
dataset_clean_slots

,bikes,bikes_ok,entry_date,slots,slots_ok,station_id,station_lat,station_lon,station_name
0,0,False,2018-07-04 11:21:08,20,True,29,41.401101,2.170082,"Provença, 388"
1,17,True,2018-07-04 11:21:08,11,True,19,41.403282,2.170726,"Rossello, 354"
2,1,False,2018-07-04 11:21:08,30,True,362,41.397006,2.170570,"Bailén, 100"
3,1,False,2018-07-04 11:21:08,18,True,123,41.397411,2.167798,"Girona, 126"
4,0,False,2018-07-04 11:21:08,24,True,224,41.399102,2.165556,"Girona, 156"
5,2,True,2018-07-04 11:21:08,27,True,370,41.403735,2.172987,"Sardenya, 234"
6,0,False,2018-07-04 11:21:08,19,True,22,41.402051,2.175206,"Sardenya, 296"


Display on a map the cleaned up dataset:

In [10]:
import folium

folmap_bikes = folium.Map(location=current_latlon, tiles='OpenStreetMap', zoom_start=20)
folmap_slots = folium.Map(location=current_latlon, tiles='OpenStreetMap', zoom_start=20)

Add to the map the markers of the available stations:

In [11]:
for idx,i in enumerate(dataset_clean_bikes.station_id):
    folium.Marker([dataset_clean_bikes.station_lat[idx], 
                   dataset_clean_bikes.station_lon[idx]], 
                   popup=dataset_clean_bikes.station_name[idx]+' / Available bikes: '+dataset_clean_bikes.bikes[idx]+' / Available slots: '+dataset_clean_bikes.slots[idx]).add_to(folmap_bikes)
folmap_bikes

In [12]:
for idx,i in enumerate(dataset_clean_slots.station_id):
    folium.Marker([dataset_clean_slots.station_lat[idx], 
                   dataset_clean_slots.station_lon[idx]], 
                   popup=dataset_clean_slots.station_name[idx]+' / Available bikes: '+dataset_clean_slots.bikes[idx]+' / Available slots: '+dataset_clean_slots.slots[idx]).add_to(folmap_slots)
folmap_slots